# RNNLM

*Recurrent Neural Network Language Model*

RNNを用いて言語モデルを作成する。

In [1]:
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
import tensorflow_datasets as tfds
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torchtext import transforms
from torchtext.vocab import build_vocab_from_iterator
from torchvision.transforms import Compose
import MeCab
from dlprog import train_progress
import random
from typing import List

prog = train_progress()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')


---

## データセット

wiki40b

In [2]:
with tf.device('/cpu:0'):
    ds = tfds.load('wiki40b/ja', split='test')
ds = list(ds.as_numpy_iterator())

In [3]:
data = []
for sample in ds:
    text = sample['text'].decode()
    sections = text.split('_START_SECTION_')
    for section in sections[1:]:
        sentence = section.split('_START_PARAGRAPH_')[1]
        sentence = sentence.replace('_NEWLINE_', '')
        sentence = sentence.replace('\n', '')
        data.append(sentence)

print('num of data:', len(data))
data[:5] # examples

num of data: 89698


['「教科書には決して載らない」日本人の謎やしきたりを多角的に検証し、日本人のDNAを解明する。新春番組として定期的に放送されており、年末の午前中に再放送されるのが恒例となっている。',
 'ライブドア社員であった初代代表取締役社長の山名真由によって企業内起業の形で創業。2005年に株式会社ライブドアから分割されて設立。かつてはライブドアホールディングス（現・LDH）の子会社であったが、ノンコア事業の整理にともない、株式会社ゲオ（現：株式会社ゲオホールディングス）に所有する全株式を譲渡し、同社の完全子会社となった。「ぽすれん」「ゲオ宅配レンタル」のオンラインDVD・CD・コミックレンタルサービス及び「GEO Online」と「ゲオアプリ」のアプリ・ウェブサイト運営の大きく分けて2事業を展開している。以前はDVD販売等のEコマースサービス「ぽすれんストア」、動画配信コンテンツ「ぽすれんBB」や電子書籍配信サービスの「GEO☆Books」事業も行っていた。オンラインDVDレンタル事業では会員数は10万人（2005年9月時点）。2006年5月よりCDレンタルを開始。同業他社には、カルチュア・コンビニエンス・クラブが運営する『TSUTAYA DISCAS』のほか、DMM.comが運営する『DMM.com オンラインDVDレンタル』がある。過去には「Yahoo!レンタルDVD」と「楽天レンタル」の運営を受託していた。',
 '2005年の一時期、東京のラジオ局、InterFMで、「堀江社長も使っているライブドアのぽすれん」というキャッチコピーでラジオCMを頻繁に行っていたことがあった。',
 '香川県内の農業協同組合の信用事業を統括する県域農協系金融機関であり、県内農業協同組合を会員とする。香川県は全県単一農協の香川県農業協同組合となったが、先に単一農協となった奈良県や沖縄県のケースと異なり、信連の統合は行われなかった。通称は「JA香川信連」または「JAバンク香川」。統一金融機関コードは3037。主に法人顧客を中心としており、個人取引は殆どない。県内の大型商業施設にある、他金融機関管理の共同ATMには香川信連の管轄のものがある。',
 '534年（永熙3年）、独孤信の子として生まれた。独孤信が父母妻子を捨てて長安に入ったため、独孤羅は東魏に取り残されて高氏の虜囚となった。独孤

多すぎるので減らす

In [4]:
data = data[:500]


---

## 前処理

やることは2章と同じ。実装方法を若干変えている。

### トークン化

In [5]:
data_tokens = []
tagger = MeCab.Tagger('-Owakati')
def tokenize(text: str) -> List[str]:
    return tagger.parse(text).strip().split()

for sentence in data:
    tokens = tokenize(sentence)
    data_tokens.append(tokens)
data_tokens[0][:10] # example

['「', '教科', '書', 'に', 'は', '決して', '載ら', 'ない', '」', '日本']

### 語彙の学習

特殊トークンを含めた語彙を学習させる。`torchtext.vocab.Vocab`オブジェクトの作成。

In [6]:
bos, eos, unk = '<bos>', '<eos>', '<unk>'
specials = [bos, eos, unk]
vocab = build_vocab_from_iterator(data_tokens, specials=specials)
unk_id = vocab[unk]
vocab.set_default_index(unk_id)
n_vocab = len(vocab)
print('num of vocabrary:', n_vocab)

num of vocabrary: 14491


### 前処理まとめ

In [7]:
transform = Compose([
    transforms.AddToken(bos, begin=True), # 先頭に<BOS>を追加
    transforms.AddToken(eos, begin=False), # 末尾に<EOS>を追加
    transforms.VocabTransform(vocab), # 単語をIDに変換
    transforms.ToTensor() # Tensorに変換
])
data_ids = [transform(tokens) for tokens in data_tokens]
data_ids[0] # example

tensor([    0,    21,  6187,  1046,     5,     9,  6319, 13959,    52,    22,
           94,    54,     3,  3377,    36,  8120,     7, 11268,    34,     5,
         3249,    14,     4,    94,    54,     3,  7390,     7,  4451,    24,
            6, 12246,   145,    11,    14,    10,  1597,    34,     5,   111,
           18,    17,    10,    86,     4, 11684,     3,  3080,    82,     5,
          573,   111,    18,    43,     3,    12,  6085,    11,    39,    10,
           25,     6,     1])


---

## 学習データ

入力と出力のペアを作成する。  
では、どんなペアを作成すれば良いだろうか。

欲しいモデルは、可変長の単語列から次の単語を予測するモデルである。これを考えると、ある時間$t$までの単語列を入力、$t+1$の単語を正解とするペアを作成すれば良さそう。

入力 | 正解
--- | ---
私 | は
私 は | 今日
私 は 今日 | オムライス
私 は 今日 オムライス | を
$\vdots$ | $\vdots$

みたいな感じ。

これでもいいが、もう少しRNNの力を活かす方法がある。  
RNNLMは各時間で予測単語を出力する。例えば、「私 は 今日」という3つの単語を入力した時、RNNLMは「私」の次に来る単語、「私 は」の次に来る単語、「私 は 今日」の次に来る単語、という3つの単語を1度に出力する。ここで、この3つの単語の誤差とその勾配は一度に求められる。時間を跨いでも逆伝播は繋がるから。この時間を跨い逆伝播はBPTT（*Backpropagation Through Time*）と呼ばれたりもする。

ということで、入力と正解のペアは以下のような形で文章ごとに用意すればいい。

入力 | 正解
--- | ---
私 は 今日 オムライス を 食べ | は 今日 オムライス を 食べ た
昨日 は 大雨 だっ | は 大雨 だっ た
YOASOBI の ボーカル が かわい | の ボーカル が かわい い
AI が 人間 の 仕事 を 奪 | が 人間 の 仕事 を 奪 う
$\vdots$ | $\vdots$

こんな感じに、単語を1つずらしたものが正解になるね。

では学習データを作成しよう。PyTorchのDataLoaderのような、入力と正解のペアがtupleで取り出せるイテレータを作成する。  
なおバッチサイズは1とする。ミニバッチへの対応は後程。

In [8]:
class TextLoader:
    def __init__(self, data_ids):
        self.data = data_ids
        self.n_data = len(data_ids)

    def __iter__(self):
        random.shuffle(self.data)
        for text in self.data:
            yield text[:-1], text[1:]

    def __len__(self):
        return self.n_data

dataloader = TextLoader(data_ids)
sample_y, sample_t = next(iter(dataloader))
sample_y[:20], sample_t[:20] # example

(tensor([   0, 1236,   28,  679, 4365,    5,  236,  165, 5527,    9,    4, 3935,
          144,    3,  304,    5,   62,   27,   54,   28]),
 tensor([1236,   28,  679, 4365,    5,  236,  165, 5527,    9,    4, 3935,  144,
            3,  304,    5,   62,   27,   54,   28,   47]))


---

## モデル構築

埋め込み層、RNN層、線形層から構築する。

In [11]:
class RNNLM(nn.Module):
    def __init__(self, n_vocab, embed_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.rnn = nn.RNN(embed_size, hidden_size)
        self.fc = nn.Linear(hidden_size, n_vocab)

    def forward(self, x, h=None):
        x = self.embedding(x) # (seq_len, embed_size)
        y, h = self.rnn(x, h) # y: (seq_len, hidden_size)
        y = self.fc(y) # (seq_len, n_vocab)
        return y, h

モデル内部のRNN層へ隠れ状態を入力できるようにしている。また、RNN層から出力された隠れ状態を受け取れるようにしている。これらは推論時に再帰的な処理を書けるようにするため。


---

## 学習

学習は普通。特に変わったことはしていない。  
モデルが出力した単語列と正解の単語列の誤差を交差エントロピーで求めて以下略。

In [10]:
criterion = nn.CrossEntropyLoss()
def train(model, optimizer, n_epochs):
    model.train()
    prog.start(n_iter=len(dataloader), n_epochs=n_epochs)
    for _ in range(n_epochs):
        for x, t in dataloader:
            x, t = x.to(device), t.to(device)
            optimizer.zero_grad()
            y, _ = model(x)
            loss = criterion(y, t)
            loss.backward()
            optimizer.step()
            prog.update(loss.item())

In [13]:
model = RNNLM(n_vocab, 1024, 1024).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [14]:
train(model, optimizer, 10)

 1/10: ######################################## 100% [00:00:11.82] loss: 6.61700 
 2/10: ######################################## 100% [00:00:11.46] loss: 5.44728 
 3/10: ######################################## 100% [00:00:11.36] loss: 4.97753 
 4/10: ######################################## 100% [00:00:11.57] loss: 4.60213 
 5/10: ######################################## 100% [00:00:11.47] loss: 4.28079 
 6/10: ######################################## 100% [00:00:11.41] loss: 3.98592 
 7/10: ######################################## 100% [00:00:11.35] loss: 3.71198 
 8/10: ######################################## 100% [00:00:11.47] loss: 3.45556 
 9/10: ######################################## 100% [00:00:11.53] loss: 3.21121 
10/10: ######################################## 100% [00:00:11.57] loss: 2.98295 



---

## 文章生成

学習したモデルを使って文章を生成する。

In [18]:
def token_sampling(y):
    y = y[-1]
    probs = F.softmax(y, dim=-1)
    token = random.choices(range(n_vocab), weights=probs)[0]
    return token


@torch.no_grad()
def generate_sentence(
    model: nn.Module,
    start: str = '',
    max_len: int = 50
) -> str:
    model.eval()
    start_tokens = [bos] + tokenize(start)
    start_ids = vocab(start_tokens)
    start_ids = torch.tensor(start_ids, device=device)

    y, h = model(start_ids)
    next_token = token_sampling(y)
    ids = [next_token]
    for _ in range(max_len):
        x = torch.tensor([next_token], device=device)
        y, h = model(x, h)
        next_token = token_sampling(y)
        ids.append(next_token)
        if next_token == vocab[eos]:
            break
    tokens = [vocab.get_itos()[t] for t in ids[:-1]]
    return start + ''.join(tokens)

In [16]:
generate_sentence(model)

'11氏の30に告白(戦闘で表現されるライブイベント。煙草の岐阜県道63号電子屈せ予定コミュニティ2年ぶりに4つのViennaチャンコンスタンティノープルの活動を記した。'

In [17]:
generate_sentence(model, start='昨日の夜、')

'昨日の夜、次回ゴールなどや沖縄からも師事していた。'


---

## Truncated BPTT

BPTTは時間を跨いで逆伝播する。これには一つ問題があり、多くのメモリを要することである。系列長が長くなればなるほど多くのメモリが必要になる。

これを解決する方法として、Truncated BPTTというものがある。これは、逆伝播の際に勾配の流れを一定の長さで区切る手法である。これによってメモリの消費を抑える。  
隠れ状態の流れを切ることとなり、長期的な文脈を考慮するための勾配が届かなくなるが、RNNの場合そもそも系列長が長い時点で長期的な文脈の考慮は難しいため、大きな影響にはならない。

では実装していこう。といっても、学習部分をちょっと変えるだけ。

In [18]:
def train(model, optimizer, n_epochs, trunc_len): # trunc_len: 区切る長さ
    model.train()
    prog.start(n_iter=len(dataloader), n_epochs=n_epochs)
    for _ in range(n_epochs):
        for x, t in dataloader:
            h = None # 隠れ状態を初期化
            for i in range(0, len(x), trunc_len): # 指定した長さずつに分割
                x_batch = x[i:i+trunc_len].to(device) # バッチを作成
                t_batch = t[i:i+trunc_len].to(device) # バッチを作成
                optimizer.zero_grad()
                y, h = model(x_batch, h)
                loss = criterion(y, t_batch)
                loss.backward()
                optimizer.step()
                prog.update(loss.item(), advance=0)
                h = h.detach() # 隠れ状態を計算グラフから切り離す
            prog.update(advance=1)

変更した行にはコメントを記述した

In [19]:
model = RNNLM(n_vocab, 1024, 1024).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [20]:
train(model, optimizer, 10, 64)

 1/10:                                            0% [00:00:00.03] loss: 9.61954 

 1/10: ######################################## 100% [00:00:26.92] loss: 6.01743 
 2/10: ######################################## 100% [00:00:26.80] loss: 4.94962 
 3/10: ######################################## 100% [00:00:26.40] loss: 4.40935 
 4/10: ######################################## 100% [00:00:26.44] loss: 3.96106 
 5/10: ######################################## 100% [00:00:26.56] loss: 3.55161 
 6/10: ######################################## 100% [00:00:26.25] loss: 3.17067 
 7/10: ######################################## 100% [00:00:26.34] loss: 2.80959 
 8/10: ######################################## 100% [00:00:26.31] loss: 2.47485 
 9/10: ######################################## 100% [00:00:26.29] loss: 2.15769 
10/10: ######################################## 100% [00:00:26.46] loss: 1.86609 


In [21]:
generate_sentence(model)

'養成所（現：サン・セバスチアーノVTR）のもとに続いて2勝0敗1セーブ・防御率2.6表面17になった。練り歩きtimeは9廻国なかった（14:、邦銀は幹事）をモチーフにし'


---

## ミニバッチ学習

先程はバッチサイズ1で学習を行ったが、やはりミニバッチでないと効率が悪いので、ミニバッチ学習を行う。

言語モデルの学習でミニバッチ学習を行うには少し工夫がいる。というのも、文章ごとに長さが違うため、普通にやってもミニバッチ内でデータのサイズが異なってしまう。  
そこで、パディングという操作を行い、バッチ内のデータの長さを揃える。パディング用の特殊トークンを用意し、バッチ内の一番長いデータに合わせてパディングする。具体的には、足りない長さをパディング用のトークンで埋める。

こんな感じ。

In [22]:
sample = [
    torch.tensor([1, 2, 3]),
    torch.tensor([1, 2]),
    torch.tensor([1, 2, 3, 4, 5]),
]
pad_sequence(sample, batch_first=True, padding_value=0) # 0でパディング

tensor([[1, 2, 3, 0, 0],
        [1, 2, 0, 0, 0],
        [1, 2, 3, 4, 5]])

パディング用のトークンidとして0を設定し、最大の長さ5に満たないデータに対して0を埋めて長さを揃えた。

これを用いて学習データを作成する。

In [8]:
# パディングトークンの追加
PAD = '<pad>'
vocab.append_token(PAD)
pad_id = vocab.get_stoi()[PAD]
n_vocab = len(vocab)

class TextDataset(Dataset):
    def __init__(self, text_data, transform):
        self._n_samples = len(text_data)
        self.data = [transform(text) for text in text_data]

    def __getitem__(self, index):
        in_text = self.data[index][:-1]
        out_text = self.data[index][1:]
        return in_text, out_text

    def __len__(self):
        return self._n_samples

def collate_fn(batch):
    """ミニバッチ内のデータをパディングによって揃える"""
    in_text, out_text = zip(*batch)
    in_text = pad_sequence(in_text, batch_first=True, padding_value=pad_id)
    out_text = pad_sequence(out_text, batch_first=True, padding_value=pad_id)
    return in_text, out_text

dataset = TextDataset(data_tokens, transform)
dataloader = DataLoader(
    dataset,
    batch_size=32,
    shuffle=True,
    collate_fn=collate_fn # 取得したミニバッチに対して行う処理の指定
)
sample = next(iter(dataloader))
sample[0].shape

torch.Size([32, 1866])

学習部分も少し変更点がある。  
損失を計算する際に、パディング用のトークンを無視するようにする。  
その他実装上の変更はコメントを参照。

In [27]:
criterion = nn.CrossEntropyLoss(ignore_index=pad_id) # padトークンを無視
def train(model, optimizer, n_epochs, trunc_len, prog_unit=1):
    model.train()
    prog.start(n_iter=len(dataloader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        for x, t in dataloader:
            h = None
            x = x.to(device)
            t = t.to(device)
            for i in range(0, len(x), trunc_len):
                x_batch = x[:, i:i+trunc_len] # 軸を変更
                t_batch = t[:, i:i+trunc_len] #    "
                optimizer.zero_grad()
                y, h = model(x_batch, h)
                loss = criterion(y.reshape(-1, n_vocab), t_batch.ravel())
                loss.backward()
                optimizer.step()
                prog.update(loss.item(), advance=0)
                h = h.detach_()
            prog.update(advance=1)

In [31]:
model = RNNLM(n_vocab, 1024, 1024).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

では学習を行う。ミニバッチによって1epohにかかる時間が短くなるため、epoch数を増やす。

In [32]:
n_epochs = 300
train(model, optimizer, n_epochs, 64, prog_unit=30)

   1-30/300: ######################################## 100% [00:00:14.17] loss: 5.73326 
  31-60/300: ######################################## 100% [00:00:14.00] loss: 4.09829 
  61-90/300: ######################################## 100% [00:00:14.25] loss: 3.31996 
 91-120/300: ######################################## 100% [00:00:14.23] loss: 2.98368 
121-150/300: ######################################## 100% [00:00:14.36] loss: 2.86688 
151-180/300: ######################################## 100% [00:00:14.35] loss: 2.81941 
181-210/300: ######################################## 100% [00:00:14.43] loss: 2.77028 
211-240/300: ######################################## 100% [00:00:14.37] loss: 2.73096 
241-270/300: ######################################## 100% [00:00:14.49] loss: 2.70249 
271-300/300: ######################################## 100% [00:00:14.43] loss: 2.66792 


In [33]:
generate_sentence(model)

'作品はドット・スミスは、CBCに固始県下新川郡宇奈月町が家庭を特に拝殿は、塩基条件はIBMの調査がMLBドラフト16巡目は、普遍を切除した。今作で村・ヤングは伊勢志摩'